<a href="https://colab.research.google.com/github/YahyaAlaaMassoud/learn-search-relevance/blob/main/quora_semantic_search_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

Creating requirments.txt file and writing the needed dependencies, then installing them.

In [2]:
%%writefile requirments.txt
datasets
gradio
qdrant-client
sentence-transformers

Writing requirments.txt


In [3]:
!echo "Requirments Are:" && cat requirments.txt && echo "-----"

!pip install -r requirments.txt

Requirments Are:
datasets
gradio
qdrant-client
sentence-transformers
-----
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.3 MB/s eta 0:00:00ta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 2.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 3.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 755.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 41.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 7.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Memory requirments for building vector index](https://qdrant.tech/documentation/cloud/capacity-sizing/)

```
memory_size = number_of_vectors * vector_dimension * 4 bytes * 1.5
```

In [7]:
from build_quora_index import build_index

In [5]:
%load_ext gradio

In [24]:
import gradio as gr
import os

from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient


qdrant = QdrantClient(':memory:') # create in-mem instance of vector db
# qdrant = QdrantClient(
#     url=os.environ['QDRANT_URL'],
#     api_key=os.environ['QDRANT_API_KEY'],
# )
encoder = SentenceTransformer(model_name_or_path='BAAI/bge-small-en-v1.5')


def compute_embedding(sentences, emb_model):
  return emb_model.encode(sentences=sentences)


def query(question, top_k=5):
  collections_names = list(map(lambda x: x.name, qdrant.get_collections().collections))
  if 'questions' not in collections_names:
    print('BUILDING INDEX')
    build_index()

  results = qdrant.search(
    collection_name='questions',
    query_vector=compute_embedding(question, encoder),
    limit=top_k
  ) # search for top K questions in the quora questions DB similar to the new question

  results_dict = {}
  for result in results:
    results_dict[result.payload['question']] = round(result.score, 3)

  return results_dict


with gr.Blocks() as semantic_search_demo:
  gr.Markdown(
    """
    # Semantic search for similar questions from Quora
    """
  )

  input_question = gr.Textbox(
    label="Question", 
    placeholder="Enter your question here"
  )
  top_k_slider = gr.Slider(
    value=3, 
    minimum=1, 
    maximum=8, 
    label="Top K", 
    interactive=True, 
    step=1
  )
  button = gr.Button("Find similar questions")
  results = gr.Label(
    label="Most similar questions", 
    value={}
  )

  gr.Examples(
    examples=[["Obama", 3]], 
    inputs=[input_question, top_k_slider]
  )

  button.click(fn=query, inputs=[input_question, top_k_slider], outputs=[results])


semantic_search_demo.launch(share=True, debug=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://e1b0371c47fda811ac.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
